In [10]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [11]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cuda:1'

In [12]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [13]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Talklife MoC

In [14]:
%run load_talklifemoc.py

In [15]:
%run load_sbert-embeddings.py

In [16]:
sbert_embeddings.shape

torch.Size([18604, 384])

SWNU Network

In [17]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
include_features_in_path = True
include_features_in_input = False

In [18]:
#create indices for kfold
fold_col_names = [c for c in df.columns if 'fold' in c ]
fold_list = []
for foldc in fold_col_names:
    fold_list.append((df[df[foldc]=='train'].index, df[df[foldc]=='dev'].index, df[df[foldc]=='test'].index))
fold_list = tuple(fold_list)

In [19]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
dimreduction_method = ["umap"]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [20]:
size = 11
(swnu_network_umap_11,
    best_swnu_network_umap_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dimreduction_method,
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=None, 
    split_indices=fold_list,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/swnu_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/swnu_network_umap_focal_2_11_kfold_best_model.csv


In [21]:
swnu_network_umap_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_206338/2148238126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.810238   
                                                       0.0005         0.797911   
                                                       0.0010         0.805920   
                                          0.2          0.0001         0.809539   
                                                       0.0005         0.800294   
                                                       0.0010         0.809414   
                           (512, 512)     0.1          0.0001         0.803662   
                                                       0.0005         0.797768   
                                                       0.0010         0.801530   
                                          0.2          0.0001         0.802193   
                                                       0.0005         0.799183   
                                                       0.0010         0.804881   
           (12,)           (256, 256)     0.1          0.0001         0.808213   
                                                       0.0005         0.802999   
                                                       0.0010         0.801405   
                                          0.2          0.0001         0.807085   
                                                       0.0005         0.801458   
                                                       0.0010         0.805920   
                           (512, 512)     0.1          0.0001         0.807031   
                                                       0.0005         0.803752   
                                                       0.0010         0.801190   
                                          0.2          0.0001         0.806816   
                                                       0.0005         0.803196   
                                                       0.0010         0.806404   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.538563   
                                                       0.0005         0.535687   
                                                       0.0010         0.541316   
                                          0.2          0.0001         0.538196   
                                                       0.0005         0.539831   
                                                       0.0010         0.541850   
                           (512, 512)     0.1          0.0001         0.535693   
                                                       0.0005         0.539451   
                                                       0.0010         0.534859   
                                          0.2          0.0001         0.536138   
                                                       0.0005         0.538247   
                                                       0.0010         0.536478   
           (12,)           (256, 256)     0.1          0.0001         0.541125   
                                                       0.0005         0.543539   
                                                       0.0010         0.546761   
                                          0.2          0.0001         0.541009   
                                                       0.0005         0.542980   
                                                       0.0010         0.546558   
                           (512, 512)     0.1          0.0001         0.540314   
                                                       0.0005         0.541419   
                                                       0.0010         0.537357   
                                          0.2      

In [22]:
best_swnu_network_umap_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.816276,0.540623,"[0.8943354076910802, 0.44810307802433785, 0.27...",0.540878,"[0.8964237732710639, 0.4317241379310345, 0.294...",0.541289,"[0.8922567498726439, 0.46577380952380953, 0.26...",None,0.815056,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.801172,0.541213,"[0.8843616986710856, 0.45161290322580644, 0.28...",0.526136,"[0.9028726862602003, 0.40717131474103585, 0.26...",0.561165,"[0.8665944982170147, 0.5069444444444444, 0.309...",None,0.794673,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.803806,0.541192,"[0.8855969207879161, 0.4464285714285714, 0.291...",0.530174,"[0.8998882534674292, 0.40584415584415584, 0.28...",0.555476,"[0.8717524197656648, 0.49603174603174605, 0.29...",None,0.799883,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [30]:
best_swnu_network_umap_11[['f1', 'f1_scores', 'precision', 
       'recall', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 
       'valid_recall', 
       'dimensions','swnu_hidden_dim','sig_depth','ffn_hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'k_fold', 
        'batch_size']]

,f1,f1_scores,precision,recall,valid_f1,valid_f1_scores,valid_precision,valid_recall,dimensions,swnu_hidden_dim,sig_depth,ffn_hidden_dim,dropout_rate,learning_rate,seed,loss_function,k_fold,batch_size
0,0.540623,"[0.8943354076910802, 0.44810307802433785, 0.27...",0.540878,0.541289,0.541262,"[0.8932130520142152, 0.46561037090385304, 0.26...",0.542319,0.542925,15,"(12,)",3,"(256, 256)",0.2,0.0001,1,focal,True,64
0,0.541213,"[0.8843616986710856, 0.45161290322580644, 0.28...",0.526136,0.561165,0.539522,"[0.8793685439455851, 0.4577755742852421, 0.281...",0.523542,0.562405,15,"(12,)",3,"(256, 256)",0.2,0.0001,12,focal,True,64
0,0.541192,"[0.8855969207879161, 0.4464285714285714, 0.291...",0.530174,0.555476,0.539421,"[0.8827474910484643, 0.45564924114671157, 0.27...",0.528380,0.555175,15,"(12,)",3,"(256, 256)",0.2,0.0001,123,focal,True,64


In [23]:
best_swnu_network_umap_11["f1"].mean()

0.5410092779610705

In [24]:
best_swnu_network_umap_11["precision"].mean()

0.5323960588604976

In [25]:
best_swnu_network_umap_11["recall"].mean()

0.5526431174439549

In [26]:
np.stack(best_swnu_network_umap_11["f1_scores"]).mean(axis=0)

array([0.88809801, 0.44871485, 0.28621497])

In [27]:
np.stack(best_swnu_network_umap_11["precision_scores"]).mean(axis=0)

array([0.89972824, 0.4149132 , 0.28254674])

In [28]:
np.stack(best_swnu_network_umap_11["recall_scores"]).mean(axis=0)

array([0.87686789, 0.48958333, 0.29147813])